## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from arch import arch_model
from scipy.optimize import minimize

## Dynamic Portfolio: <span style="font-size:20px; color:white; font-weight:900">Risk Parity (Strategy) + GARCH (Simulation)</span>

In [ ]:


# Step 1: Download historical data
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']  # Example assets
data = yf.download(tickers, start='2015-01-01', end='2025-01-01')['Adj Close']
returns = np.log(data / data.shift(1)).dropna()

# Step 2: Fit GARCH(1,1) model and forecast volatility
def garch_volatility_forecast(returns, horizon=1):
    model = arch_model(returns, vol='Garch', p=1, q=1)
    res = model.fit(disp='off')
    forecast = res.forecast(start=returns.index[-1], horizon=horizon, reindex=False)
    return np.sqrt(forecast.variance.iloc[-1])  # Annualized volatility

# Compute volatility forecasts
vol_forecasts = pd.Series({ticker: garch_volatility_forecast(returns[ticker]) for ticker in tickers})

# Step 3: Compute Risk Parity Weights
def risk_parity_weights(vols):
    inv_vols = 1 / vols
    weights = inv_vols / inv_vols.sum()
    return weights

weights = risk_parity_weights(vol_forecasts)

# Step 4: Portfolio Rebalancing Function
def rebalance_portfolio(returns, rebalance_freq='M'):
    weight_history = []
    
    for date, ret in returns.resample(rebalance_freq).mean().iterrows():
        vol_forecasts = pd.Series({ticker: garch_volatility_forecast(returns[ticker].loc[:date]) for ticker in tickers})
        new_weights = risk_parity_weights(vol_forecasts)
        weight_history.append(pd.Series(new_weights, name=date))
    
    return pd.DataFrame(weight_history)

# Compute dynamic weights over time
dynamic_weights = rebalance_portfolio(returns)

# Display final portfolio weights
dynamic_weights.tail()
